In [1]:
import pandas as pd

In [2]:
import regex as re

In [3]:
df = pd.read_pickle('THINK_TANKS_UN_CORPUS_INTERSECTION_VERB_NOUN_PAT.pkl')

In [4]:
df = df['ar	en	AR_LEN	EN_LEN	PAT_LEMMA	PAT_TEXT'.split()]

In [5]:
quiz_01 = df.sample(100_000)

In [6]:
quiz_01

,ar,en,AR_LEN,EN_LEN,PAT_LEMMA,PAT_TEXT
143109,:: في حين أن تعليم كل من الفتيان والفتيات يؤدي...,While education for both boys and girls leads ...,21,23,increase income,increased income
37789,وﻻحظ أن الفقرة الفرعية )ز( من الفقرة ١ في اطار...,He noted that subparagraph (g) of paragraph 1 ...,20,18,mention the need,mentioned the need
294213,وستواصل تعزيز حقوق الإنسان بالرغم من انتقادها ...,"It would continue to promote human rights, des...",10,14,promote human right,promote human rights
353659,ومن المتوقع أن تدخل الاتفاقية حيز النفاذ في عا...,It was expected that the Convention would come...,10,12,come into force,come into force
260742,9 - الرئيس وجه الانتباه إلى المبادئ التوجيهية ...,9. The Chairman drew attention to the guidelin...,20,21,draw attention,drew attention
...,...,...,...,...,...,...
107255,36- وأكّد مندوب آخر أهمية إشراك المجتمع المدني...,Another delegate underlined the importance of ...,17,21,underline the importance,underlined the importance
295914,وهذا العامل ليس مشكلة تتفرد بها تونغا، فهو شاغ...,This is not just a problem for Tonga but is a ...,17,18,share concern,shared concern
167395,- الإحاطة علما بأن الدول المشاركة في المبادئ ا...,takes note that the States participating in th...,14,13,take note,takes note
340708,ويتألف المؤشر من صادرات السلع وحدها، على الرغم...,"The indicator comprises goods only, although e...",18,16,develop country,developed countries


In [7]:
def verb_gap(x):
    result = x.split()
    result[0] = '#'*12
    result = ' '.join(result)
    return result

In [8]:
def remove_verb(x0, x1):
    try:
        result = re.split(f'({x0})', x1) # for what?
        idx = [(i,j) for i,j in enumerate(result) if j == x0]
        idx = idx[0][0]
        result[idx] = verb_gap(result[idx])
        result = ' '.join(result)
#         print(result)
        return result
    
    except:
        print('ERROR!')
        return 'error!'

In [9]:
quiz_01['QUESTION'] = quiz_01.apply(lambda x: remove_verb(x['PAT_TEXT'], x['en']), axis=1)

In [10]:
quiz_01 = quiz_01.groupby('PAT_LEMMA').agg({
    'QUESTION': lambda x: x.tolist()[:10],
    'PAT_TEXT': lambda x: x.tolist()[:10],
    'PAT_LEMMA': lambda x: x.tolist()[:10],
    'EN_LEN': lambda x: x.tolist()[:10],
    'AR_LEN': lambda x: x.tolist()[:10],
    'ar': lambda x: x.tolist()[:10],
    'en': lambda x: x.tolist()[:10]
    
    
})

In [11]:
quiz_01 = quiz_01.explode('QUESTION	PAT_TEXT	PAT_LEMMA	EN_LEN	AR_LEN	ar	en	'.split())

In [12]:
quiz_01 = quiz_01.drop_duplicates(['QUESTION']).reset_index(drop=True)

In [13]:
print(quiz_01['QUESTION'].iloc[1])

(d) Must  ############ by the rules  of the organ executing the control for meeting visitors;


In [14]:
print(quiz_01['en'].iloc[1])

(d) Must abide by the rules of the organ executing the control for meeting visitors;


In [15]:
quiz_01 = quiz_01[quiz_01['PAT_TEXT'].apply(lambda x: len(x.split())>1)]

In [16]:
quiz_01 = quiz_01.reset_index(drop=True)

In [17]:
quiz_01['VERB'] = quiz_01['PAT_TEXT'].apply(lambda x: x.split()[0])

In [18]:
quiz_01['VERB_LEMMA'] = quiz_01['PAT_LEMMA'].apply(lambda x: x.split()[0])

In [19]:
quiz_01['NOUN'] = quiz_01['PAT_TEXT'].apply(lambda x: x.split()[-1])

In [20]:
quiz_01 = quiz_01[quiz_01['VERB_LEMMA'].apply(lambda x: x.lower() != 'be')]

In [21]:
quiz_01 = quiz_01.reset_index(drop=True)

In [22]:
quiz_01.sample(30)

,QUESTION,PAT_TEXT,PAT_LEMMA,EN_LEN,AR_LEN,ar,en,VERB,VERB_LEMMA,NOUN
8390,He believed that he ############ the support ...,had the support,have the support,14,12,وأعرب عن اعتقاده بأن البيان المشار إليه أعلاه ...,He believed that he had the support of all Par...,had,have,support
9572,In order to eradicate poverty and ###########...,increase prosperity,increase prosperity,21,21,١٨- وقال إنَّ القضاء على الفقر وتحقيق مزيد من ...,In order to eradicate poverty and increase pro...,increase,increase,prosperity
16016,Hybrid approaches often support gradual transi...,share the burden,share the burden,19,20,وكثيراً ما تدعم النُهج الهجينة الانتقال التدري...,Hybrid approaches often support gradual transi...,share,share,burden
971,52. It was encouraging that new ground- ######...,based research,base research,12,10,٥٢- وممَّا يدعو إلى التشجيع إقامة مرافق أرضية ...,52. It was encouraging that new ground-based r...,based,base,research
12369,52. The Group ############ the importance of...,noted the importance,note the importance,18,16,52 - وأشار الفريق إلى أهمية بناء قدرات الدول ف...,52. The Group noted the importance of capacity...,noted,note,importance
15603,He hoped that the Council would agree on a con...,send a signal,send a signal,19,15,وتمنى أن يتوصل مجلس الأمن بتوافق الآراء إلى قر...,He hoped that the Council would agree on a con...,send,send,signal
11577,Such focus will enable the Office to ########...,make significant progress,make significant progress,22,21,وسيمكّن هذا التركيز المكتب من إحراز تقدم ملموس...,Such focus will enable the Office to make sign...,make,make,progress
16957,80. Tunisia had also ############ a number o...,taken a number,take a number,21,15,80 - واتخذت تونس كذلك عدداً من الخطوات المحددة...,80. Tunisia had also taken a number of specifi...,taken,take,number
4872,Recent trends have shown that rebel groups hav...,fighting forces,fight force,14,12,وتبرز اﻻتجاهات الحديثة بأن المجموعات المتمردة ...,Recent trends have shown that rebel groups hav...,fighting,fight,forces
5135,31. Information regarding ############ hours ...,flying hours,fly hour,16,17,٣١ - وترد المعلومات المتعلقة بساعات الطيران، و...,"31. Information regarding flying hours, hire c...",flying,fly,hours


In [23]:
def clean_question(df):
    v = df['VERB']
    v_len = len(v)
    new = '_ ' * v_len
    old = r'#+'
    question = df['QUESTION']
    result =  re.sub(old, new, question)
    result = re.sub(r'\s{2,}', ' ', result)
    return result.strip()
    

In [24]:
quiz_01['QUESTION'] = quiz_01.apply(lambda x: clean_question(x), axis=1)

In [30]:
quiz_01['VERB'].iloc[9615]

'increasing'

In [32]:
#quiz_01.to_pickle('STAGE_01_18K_quiz.pkl', protocol=4)

In [33]:
quiz_01

,QUESTION,PAT_TEXT,PAT_LEMMA,EN_LEN,AR_LEN,ar,en,VERB,VERB_LEMMA,NOUN
0,Provisions on transparency will help to encour...,abide by the rules,abide by the rule,17,13,وستساعد الأحكام المتعلقة بالشفافية في تشجيع ال...,Provisions on transparency will help to encour...,abide,abide,rules
1,(d) Must _ _ _ _ _ by the rules of the organ e...,abide by the rules,abide by the rule,15,9,)د( التقيد بقوانين الهيئة المنفذة للرقابة بالن...,(d) Must abide by the rules of the organ execu...,abide,abide,rules
2,Disciplinary action has been taken against 109...,abide by the rules,abide by the rule,21,13,واتُّخذت إجراءات تأديبية بحق 109 أفراد عسكريين...,Disciplinary action has been taken against 109...,abide,abide,rules
3,"23. Everyone, including the courts and other c...",abide by the rule,abide by the rule,24,22,23- واحترام القاعدة المنصوص عليها في الفصل 32 ...,"23. Everyone, including the courts and other c...",abide,abide,rule
4,- Assist the country to reform its prison syst...,abide by the rule,abide by the rule,18,10,- مساعدة البلد على إصلاح نظام سجونه والالتزام ...,- Assist the country to reform its prison syst...,abide,abide,rule
...,...,...,...,...,...,...,...,...,...,...
18575,United States officials reportedly provided _ ...,written questions,write question,20,16,ويُذكر أن مسؤولين أمريكيين قدموا أسئلة مكتوبة ...,United States officials reportedly provided wr...,written,write,questions
18576,49. Argentina thanked Grenada's for its detail...,written questions,write question,24,20,وشكرت الأرجنتين غرينادا على إجاباتها المفصلة ع...,49. Argentina thanked Grenada's for its detail...,written,write,questions
18577,(a) Initial presentation of the national repor...,written questions,write question,12,8,(أ) العرض الأولي للتقرير الوطني/الإجابة عن الأ...,(a) Initial presentation of the national repor...,written,write,questions
18578,The Uzbek delegation answered to the _ _ _ _ _...,written questions,write question,13,11,وردَّ وفد أوزبكستان على الأسئلة المكتوبة المقد...,The Uzbek delegation answered to the written q...,written,write,questions


In [34]:
pd.read_pickle('IISS_TOKEN_VS_LEMMA_78K.pkl')

,TOKEN,LEMMA
0,well,well
1,thank,thank
2,you,you
3,very,very
4,much,much
...,...,...
78076,don'ts,don't
78077,Cepeda,Cepeda
78078,semaphorin,semaphorin
78079,Rajendra,Rajendra
